# 1. Import libs

In [ ]:
# Common lib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Utils
from tqdm import tqdm
import datetime

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, confusion_matrix, precision_recall_curve

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, Flatten, AveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

print('Import successfully!')

# 2. Prepare data

In [ ]:
# Init variables
input_folder = '../input/coronahack-chest-xraydataset'
test_img_folder = os.path.join(input_folder, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'test')
train_img_folder = os.path.join(input_folder, 'Coronahack-Chest-XRay-Dataset', 'Coronahack-Chest-XRay-Dataset', 'train')
metadata_df = pd.read_csv(os.path.join(input_folder, 'Chest_xray_Corona_Metadata.csv'), index_col=0)

In [ ]:
# Split to train & test set
train_df = metadata_df[metadata_df.Dataset_type == 'TRAIN'].reset_index(drop=True)
test_df = metadata_df[metadata_df.Dataset_type == 'TEST'].reset_index(drop=True)

# Check train_df size + test_df size == metadata_df size
assert train_df.size + test_df.size == metadata_df.size

print(f'Shape of train data: { train_df.shape }')
print(f'Shape of test data: { test_df.shape }')

train_df.sample(10)

In [ ]:
# fill na
train_df.fillna('unknow', inplace=True)
test_df.fillna('unknow', inplace=True)

In [ ]:
train_datagen = ImageDataGenerator(rotation_range=10,
                              brightness_range=(0.8, 1.2),
                              horizontal_flip=True,
                              zoom_range=[0.75, 1])
test_datagen = ImageDataGenerator()

In [ ]:
train_df, valid_df = train_test_split(train_df, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
train_batches = train_datagen.flow_from_dataframe(train_df,
                                             directory=train_img_folder,
                                             x_col='X_ray_image_name',
                                             y_col='Label',
                                             class_mode='binary',
                                             batch_size=128)

valid_batches = test_datagen.flow_from_dataframe(valid_df,
                                             directory=train_img_folder,
                                             x_col='X_ray_image_name',
                                             y_col='Label',
                                             class_mode='binary',
                                             batch_size=128)

test_batches = test_datagen.flow_from_dataframe(test_df,
                                            directory=test_img_folder,
                                            x_col='X_ray_image_name',
                                            y_col='Label',
                                            class_mode='binary',
                                            batch_size=8,
                                            shuffle=False)

In [ ]:
print(f'Label encode: { train_batches.class_indices }')

In [ ]:
train_batches_series = pd.Series(train_batches.classes)
valid_batches_series = pd.Series(valid_batches.classes)

print(f'Value count in train_batches: \n{ train_batches_series.value_counts() }')
print(f'Value count in valid_batches: \n{ valid_batches_series.value_counts() }')

## 3. Fine tuning 

In [ ]:
def create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.mkdir(dir_path)
        
create_dir('models')

**Preprocessing layer**

In [ ]:
resize_and_rescale = Sequential([
    Resizing(224, 224),
    Rescaling(1./255)
])

**Initialize metric**

In [ ]:
metrics = [TruePositives(name='TP'),
           TrueNegatives(name='TN'),
           FalsePositives(name='FP'),
           FalseNegatives(name='FN'),
           AUC(curve='PR', name='AUC')]

In [ ]:
pretrained_vgg16 = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [ ]:
# Preprocessing layer
ft_vgg16 = Sequential([resize_and_rescale])
# Feature extractor layer
ft_vgg16.add(pretrained_vgg16)
# Classifier layer
ft_vgg16.add(Dense(1, activation='sigmoid'))

In [ ]:
# Freeze
#for layer in ft_vgg16.layers[:3]:
#    layer.trainable = False

In [ ]:
model_dir = 'models/resnet16'
model_file = 'best_resnet16.hdf5'

create_dir(model_dir)

checkpoint = ModelCheckpoint(os.path.join(model_dir, model_file),
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=False)

early_stopping = EarlyStopping(monitor='val_loss',
                               patience=30,
                               verbose=1,
                               restore_best_weights=True)

In [ ]:
epochs = 50
lr = 1e-4

ft_vgg16.compile(optimizer=Adam(lr=lr), loss='binary_crossentropy', metrics=metrics)

training_time_start = datetime.datetime.now()

vgg16_history = ft_vgg16.fit(train_batches,
                                epochs=epochs,
                                verbose=2,
                                callbacks=[checkpoint, early_stopping],
                                validation_data=valid_batches,
                                steps_per_epoch=len(train_batches),
                                validation_steps=len(valid_batches))

training_time_end = datetime.datetime.now()

In [ ]:
total_training_seconds = (training_time_end - training_time_start).seconds
print('Total training time: ', str(datetime.timedelta(seconds=total_training_seconds)))

## 4. Evaluate

In [ ]:
vgg16_hist_df = pd.DataFrame(vgg16_history.history)

**Plot over entire epochs**

In [ ]:
vgg16_hist_df.loc[:, ['loss', 'val_loss']].plot()
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
vgg16_hist_df.loc[:, ['AUC', 'val_AUC']].plot()
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.show()

**Plot over half first and first last epochs**

In [ ]:
num_of_epochs = vgg16_hist_df.shape[0]
half_epoch = int(num_of_epochs / 2)

first_half_vgg16_hist = vgg16_hist_df.loc[:half_epoch]
first_title = f'Loss value at epoch 0 - { half_epoch }'

last_half_vgg16_hist = vgg16_hist_df.loc[half_epoch:len(vgg16_hist_df)]
last_title = f'Loss value at epoch { half_epoch } - { len(vgg16_hist_df) }'

hists = [first_half_vgg16_hist, last_half_vgg16_hist]
titles = [first_title, last_title]

for i in range(2):
    ax = hists[i][['loss', 'val_loss']].plot()
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss value')
    ax.set_title(titles[i])
plt.show()

In [ ]:
first_title = f'AUC value at epoch 0 - { half_epoch }'
last_title = f'AUC value at epoch { half_epoch } - { len(vgg16_hist_df) }'

titles = [first_title, last_title]

for i in range(2):
    ax = hists[i][['AUC', 'val_AUC']].plot()
    ax.set_xlabel('Epoch')
    ax.set_ylabel('AUC value')
    ax.set_title(titles[i])
plt.show()

In [ ]:
evaluate_vgg16 = ft_vgg16.evaluate(test_batches, verbose=1)

In [ ]:
loss, tp, tn, fp, fn, auc = evaluate_vgg16[0], evaluate_vgg16[1], evaluate_vgg16[2], evaluate_vgg16[3], evaluate_vgg16[4], evaluate_vgg16[5]
print(f'Test loss: { loss }')
print(f'True positive: { tp }')
print(f'True negative: { tn }')
print(f'False positive: { fp }')
print(f'False negative: { fn }')
print('AUC: %.2f' % auc)

## 5. Plot PR curve and find optimal threshold

In [ ]:
def find_optimal_threshold(precision, recall, threshold):
    f1_score = (2 * precision * recall) / (precision + recall)
    best_idx = np.argmax(f1_score)
    best_threshold = threshold[best_idx]
    return best_threshold, best_idx

**Find the best threshold**

In [ ]:
y_true = test_batches.classes
y_predict = ft_vgg16.predict(test_batches)
precision, recall, threshold = precision_recall_curve(y_true, y_predict)
best_threshold, best_idx = find_optimal_threshold(precision, recall, threshold)
print('Best threshold: {}'.format(best_threshold))

**Plot ROC curve**

In [ ]:
import numpy as np
from sklearn import metrics
plt.figure(figsize=(7, 5))
auc_score = metrics.auc(recall, precision)
plt.plot([1, 0], [0, 1], linestyle='--', color='black', label='No skill')
plt.plot(recall, precision, linewidth=3, label='ResNet18')
plt.plot(recall[best_idx], precision[best_idx], 
         marker='o', color='black', 
         label='Best_theshold', linestyle='', markersize='7')
plt.xlabel('Recall', size=13)
plt.ylabel('Precision', size=13)
plt.title('Precision-Recall curve (AUC - {:.4f})'.format(auc_score), size=15)
plt.legend()
plt.show()

## 6. Predict and plot confusion matrix

In [ ]:
y_predict = (y_predict >= best_threshold).astype('int')
y_predict = np.reshape(y_predict, -1)
cfs_matrix = confusion_matrix(y_true, y_predict)

In [ ]:
label = ['Normal', 'Covid']

plt.figure(figsize=(6, 5))
plt.imshow(cfs_matrix, cmap=plt.cm.Accent)
plt.colorbar()
for i in range(len(label)):
    for j in range(len(label)):
        plt.text(j, i, cfs_matrix[i, j],
                 horizontalalignment='center', verticalalignment='center', size=14)
plt.xticks(np.arange(len(label)), label)
plt.yticks(np.arange(len(label)), label)
plt.xlabel('Predicted label', size=13)
plt.ylabel('True label', size=13)
plt.title('Confusion matrix of ResNet18', size=15)
plt.show()

In [ ]:
loss, tp, tn, fp, fn, auc = evaluate_vgg16[0], evaluate_vgg16[1], evaluate_vgg16[2], evaluate_vgg16[3], evaluate_vgg16[4], evaluate_vgg16[5]
print(f'Test loss: { loss }')
print(f'True positive: { tp }')
print(f'True negative: { tn }')
print(f'False positive: { fp }')
print(f'False negative: { fn }')
print('AUC: %.2f' % auc)